### Import libraries and define subject path

In [1]:
# This is our preprocessing script.
# We select one recording, clean stuff and then perform the ICA
# During this process, we save all bad components segments and channels

import sys
import os.path as op
module_path = op.abspath(op.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import mne
from autoreject import AutoReject, get_rejection_threshold

from data_analysis.functions_preprocessing import \
    (split_raws, mark_bads, save_bads, run_ica, save_ica,
     save_autoreject)
from data_analysis.functions_behavioral import \
    (create_event_df, remove_ghost_triggers, calculate_alpha,
     join_event_dfs, remove_outliers, events_from_event_df)

subject_dir = '/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/'
behav_dir = "/net/store/nbp/projects/hyperscanning/study_project/NBP_Hyperscanning/data_analysis/Behavioural_Analysis/BehaviouralData"


%matplotlib widget
import matplotlib
matplotlib.get_backend()


Bad key "text.kerning_factor" on line 4 in
/net/store/nbp/projects/hyperscanning/study_project/programming_tools/miniconda3/envs/hyperscanning/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


'module://ipympl.backend_nbagg'

## Define the subject you want to clean

In [2]:
subj_pair = input("Please Type in, which subject pair you want to clean.\n"
                  "For the pilot study, possible choices are:\n"
                  "[202, 203, 204, 205, 206, 207, 208, 209, 211, 212]\n")

participant = input("\nPlease Type in, which subject pair you want to clean.\n"
                    "Type: 0 for the first participant and: 1 for the second.\n")

Please Type in, which subject pair you want to clean.
For the pilot study, possible choices are:
[202, 203, 204, 205, 206, 207, 208, 209, 211, 212]
208

Please Type in, which subject pair you want to clean.
Type: 0 for the first participant and: 1 for the second.
1


## Load and prepare the EEG recording

In [3]:
# define the subjects id and its path
subj_id = "sub-{0}_p-{1}".format(subj_pair, participant)
subs_path = subject_dir + "sub-{0}/eeg/sub-{0}_task-hyper_eeg.fif".format(subj_pair)
behav_path = op.join(behav_dir, "{0}.csv".format(subj_pair))

## overwrite it for the test
# TODO: This line should be removed for the actual cleaning
#subj_id = "test_2"

# load the data
combined_raw = mne.io.read_raw_fif(subs_path, preload=True)

# split the subjects and delete the raw file
raw = split_raws(combined_raw)[int(participant)]
del combined_raw

# set reference
raw.set_eeg_reference(["Cz"])

# set the EEG Montage. We use 64 chans from the standard 10-05 system.
montage = mne.channels.make_standard_montage("standard_1005")
raw.set_montage(montage)

# filter
raw.filter(l_freq=0.1, h_freq=120)
raw.notch_filter(freqs=[50]) # bandstop the power grid

Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-208/eeg/sub-208_task-hyper_eeg.fif...


<ipython-input-3-b18a89411fb0>:11: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-208/eeg/sub-208_task-hyper_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  combined_raw = mne.io.read_raw_fif(subs_path, preload=True)


Isotrak not found
    Range : 0 ... 3725311 =      0.000 ...  3637.999 secs
Ready.
Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-208/eeg/sub-208_task-hyper_eeg-1.fif...
Isotrak not found
    Range : 3725312 ... 4035781 =   3638.000 ...  3941.192 secs
Ready.
Reading 0 ... 4035781  =      0.000 ...  3941.192 secs...
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB

<Raw | sub-208_task-hyper_eeg.fif, 80 x 4035782 (3941.2 s), ~2.41 GB, data loaded>

## Define events and epoch the data

In [4]:
# do the behavioral analysis and get the epochs
behavioral_df = calculate_alpha(pd.read_csv(behav_path))
event_df = create_event_df(raw)
event_df = remove_ghost_triggers(event_df)
event_df = join_event_dfs(event_df, behavioral_df)

# get the first tap by looking at the first sample in each trial
min_idx = event_df.groupby(["trial"])["sample"].idxmin()
early_df = event_df[event_df.index.isin(min_idx)]
early_events = events_from_event_df(early_df)

# get the late taps by looking at the last sample - 1.5 seconds
max_idx = event_df.groupby(["trial"])["sample"].idxmax()
late_df = event_df[event_df.index.isin(max_idx)]
late_events = events_from_event_df(late_df)
late_events[:,0] -= int(raw.info["sfreq"] * 1.5)

# get the baseline events
base_events = mne.pick_events(mne.find_events(raw, shortest_event=1),
                          include=48)

# define the parameters for epoching

tmin = 0
tmax = 1.5



6650 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 48 49]
6650 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 48 49]


## Run autoreject on the data

In [5]:
epochs_list = []
reject_list = []
for index, (key, events) in enumerate({"baseline":base_events,
                                       "early":early_events,
                                       "late":late_events}.items()):

    epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmax,
                        baseline=None, preload=True)  #baseline=(0, 0)

    picks = mne.pick_types(epochs.info, eeg=True)

    # define an autoreject object
    ar = AutoReject(consensus=[0.1, 0.2, 0.3, 0.4, 0.5], thresh_method="random_search",
                    picks=picks, verbose="tqdm_notebook") # [0.1, 0.2, 0.3, 0.4, 0.5], "bayesian_optimization"

    # fit the epochs
    ar.fit(epochs)

    # get the rejection threshold for ICA
    reject = get_rejection_threshold(epochs)

    # plot it
    reject_log = ar.get_reject_log(epochs)
    reject_log.plot()
    
    # plot the rejected epochs
    scalings = dict(eeg=12e-5, eog=150e-6, misc=1e-3)
    reject_log.plot_epochs(epochs, scalings=scalings)
    print("Rejected {} out of {} epochs.".format(sum(reject_log.bad_epochs), len(epochs)))
    
    # save the autoreject
    save_autoreject(ar, subj_id + "-" + key)
    
    # remove the bad epochs and add them to the epochs list
    epochs_list.append(epochs[~reject_log.bad_epochs])
    
    # add the rejects to the reject list
    reject_list.append(reject)

300 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 300 events and 1537 original time points ...
0 bad epochs dropped
Running autoreject on ch_type=eeg








Estimated consensus=0.40 and n_interpolate=4
Estimating rejection dictionary for eeg
Estimating rejection dictionary for eog


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Rejected 1 out of 300 epochs.
300 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 300 events and 1537 original time points ...
0 bad epochs dropped
Running autoreject on ch_type=eeg


...

Rejected 0 out of 300 epochs.
300 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 300 events and 1537 original time points ...
0 bad epochs dropped
Running autoreject on ch_type=eeg








Estimated consensus=0.30 and n_interpolate=1
Estimating rejection dictionary for eeg
Estimating rejection dictionary for eog


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Rejected 57 out of 300 epochs.


## Concatenate the autoreject cleaned epochs and their average reject thresholds

In [6]:
def dict_key_mean(dict_list):
    """Calculate the mean value for each key between multiple dicts.
    To return correct results, each key must be present in all of the
    dicts in dict_list."""
    import collections, functools, operator 
    # sum the values with same keys 
    sum_dict = dict(functools.reduce(operator.add, 
                                     map(collections.Counter, dict_list)))
    return {key: val/len(dict_list) for key, val in sum_dict.items()}


rejects = dict_key_mean(reject_list)
print("All reject dicts: ", reject_list)
print("Average reject dict: ", str(rejects))

epochs = mne.concatenate_epochs(epochs_list)

All reject dicts:  [{'eeg': 0.00021671228413111993, 'eog': 0.00024776855406194453}, {'eeg': 8.688310894418934e-05, 'eog': 0.00021952148382076153}, {'eeg': 0.00024082974662850397, 'eog': 0.00017150537066074056}]
Average reject dict:  {'eeg': 0.00018147504656793774, 'eog': 0.00021293180284781554}
842 matching events found
No baseline correction applied
Not setting metadata
0 bad epochs dropped


## Run (or load) the ICA and plot all components

In [7]:
# filter again for ICA
epochs.filter(l_freq=2, h_freq=None)


# run the ICA and save the marked components
picks = list(mne.pick_types(epochs.info, eeg=True, exclude=["Cz"]))
ica = run_ica(epochs, subj_id, picks=picks, reject=reject,
              n_components=63, method="fastica")

Setting up high-pass filter at 2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Filter length: 1691 samples (1.651 sec)



<ipython-input-7-807e1749c60d>:2: RuntimeWarning: filter_length (1691) is longer than the signal (1537), distortion is likely. Reduce filter length or filter a longer signal.
  epochs.filter(l_freq=2, h_freq=None)


Fitting ICA to data using 63 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Selecting by number: 63 components
Fitting ICA took 209.2s.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Get ICA components based on their correlation with EOG

In [8]:
eog_name = "BIP1" if (participant == "0") else "BIP5"
eog_idx, eog_scores = ica.find_bads_eog(epochs, ch_name=eog_name)

# barplot of ICA component "EOG match" scores
ica.plot_scores(eog_scores)

# plot diagnostics
ica.plot_properties(epochs, picks=eog_idx)


Using channel BIP5 as EOG channel


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

    Using multitaper spectrum estimation with 7 DPSS windows
842 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


[<Figure size 700x600 with 6 Axes>]

## Choose specific component properties to inspect

In [10]:
inp = input("Please type in which components you want to further "
            "inspect.\nE.G. 3, 4,15 for components 3, 4, and 15.\n")

inp = [int(n) for n in inp.split(",") if n != ""]

if len(inp) > 0:
    ica.plot_properties(epochs, picks=inp, reject=None)


Please type in which components you want to further inspect.
E.G. 3, 4,15 for components 3, 4, and 15.
20


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

    Using multitaper spectrum estimation with 7 DPSS windows
648 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


## Exclude specific ICA components

In [9]:
print("Excluded ICA components: ", ica.exclude)

inp = input("\nPlease type in which components you want to exclude.\n"
            "E.G. 2, 3,14 for components 2, 3, and 14.\n")

bad_comps = [int(comp) for comp in inp.split(",") if comp != ""]
bad_comps = [comp for comp in set(bad_comps) if comp not in ica.exclude]

ica.exclude.extend(bad_comps)
print("\nExcluded ICA components: ", ica.exclude)

Excluded ICA components:  []

Please type in which components you want to exclude.
E.G. 2, 3,14 for components 2, 3, and 14.
1

Excluded ICA components:  [1]


## Save the ICA and its excluded components

In [10]:
inp = input("Do you really want to save the components?\n"
            "Enter 'save' or 's' to save the data. Else, "
            "changes will be discarded.\n")

if inp[0] == "s":
    save_ica(ica, subj_id)

Do you really want to save the components?
Enter 'save' or 's' to save the data. Else, changes will be discarded.
s
Writing ICA solution to /net/store/nbp/projects/hyperscanning/study_project/dirk/NBP_Hyperscanning/data_analysis/bads/bad_components/sub-208_p-1-ica.fif...


### give everyone access to the new marked files you've created

In [11]:
!cd /net/store/nbp/projects/hyperscanning/study_project
!chown -hR $USER:nbp *; chmod -R 770 *



### Everything done. Thanks for cleaning :)